In [11]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(streamingpool, 1, 3, Finished, Available, Finished)

In [12]:
gold_schema = StructType([
    StructField("temperature", IntegerType()),
    StructField("humidity", IntegerType()),
    StructField("city", StringType()),
    StructField("timestamp", TimestampType()),
    StructField("date", DateType()),
    StructField("hour", IntegerType())
])

StatementMeta(streamingpool, 1, 4, Finished, Available, Finished)

In [15]:
weather_df = (spark.readStream
    .format("delta")
    .load("abfss://gold@streamingsa.dfs.core.windows.net/EventHub/weather")
)

aggregated_df = weather_df.groupBy("city", "date", "hour").agg(
    {"temperature": "mean",
    "humidity": "mean"}
    )

(aggregated_df.writeStream
    .option("checkpointLocation", "abfss://checkpoint@streamingsa.dfs.core.windows.net/checkpoint3")
    .outputMode("complete")
    .format("delta")
    .option("path", "abfss://gold@streamingsa.dfs.core.windows.net/weather_aggregated")
    .start()
)


StatementMeta(streamingpool, 1, 7, Finished, Available, Finished)